In [1]:
import pandas as pd
import numpy as np

## Reading data

`distinct_users_day.csv` contains the information about the visitors and their origin. 

`codici_istat_provincia.csv` contains the name and the code of every Italian 'provincie'. 

`Veneto.txt` is a matrix with the distances beteen all the Italian 'comuni'.

`codici_istat_comune.csv` contains the name and the code of every Italian's 'comune' and the code of its 'provincia'.

We want to compute the distribution of the visitors as a function of the distance of their origin 'provincia' with respect 
to Padova.

In this firts section we will compute the distances from all the italian 'provinice' and Padova. 
Than we will build a dataframe containig all the visitors from all the 'provincie'.

In [2]:
path="data/distinct_users_day.csv"
dist_users_day=pd.read_csv(path,sep=",",encoding="latin-1")
dist_users_day=dist_users_day[dist_users_day['CUST_CLASS']=='visitor'] #keep only visitors
dist_users_day=dist_users_day[['COD_PRO','VISITORS']].reset_index(drop=True) #keep relevant columns

In [3]:
path="data/codici_istat_provincia.csv"
codist_prov=pd.read_csv(path,sep=",",encoding="latin-1")
codist_prov=codist_prov[['COD_PRO','PROVINCIA']].reset_index(drop=True) #same as before
codist_prov=codist_prov[codist_prov.COD_PRO!=-999]                         #removing others

```bash
cat Veneto.txt | tr ',' '.' > Veneto_fixed.txt
```

We used this bash command to replace commas to points in order to avoid conversion problems

In [4]:
url='https://www.dropbox.com/s/joockt9l4az3euk/Veneto_fixed.txt?dl=1'
codist_ven=pd.read_csv(url,sep=";")
codist_ven=codist_ven[['Origine','Destinazione','Total_Mete']]

In [5]:
path="data/codici_istat_comune.csv"
codist_com=pd.read_csv(path,sep=",",encoding="latin-1")

In the next two blocks we replace the COD_COM with the COD_PRO in the distances matrix.
The problem here is that `codici_istat_comune` does not have all the COMUNI. We avoid this problem by replacing the 
missing COD_COM with the next known COD_COM.

In [6]:
dic={}
for index, row in codist_com.iterrows():
    dic[row.PRO_COM]=row.COD_PRO

def com_to_prov(c):
    try:
        p=dic[c]
    except:
        p=com_to_prov(c+1)
    return p
com_to_prov=np.vectorize(com_to_prov)

In [7]:
origine=codist_ven.Origine.astype(int).values
destinazione=codist_ven.Destinazione.astype(int).values



print(origine[:2])
origine=com_to_prov(origine)
print(origine[:2])

destinazione=com_to_prov(destinazione)

[1042 1042]
[1 1]


In [8]:
cod_Padova=codist_prov[codist_prov.PROVINCIA=='Padova'].COD_PRO.values[0]
codist_ven['Origine']=origine
codist_ven['Destinazione']=destinazione

codist_ven=codist_ven[codist_ven.Destinazione==cod_Padova]
codist_ven=codist_ven.groupby(['Origine','Destinazione'],as_index=False).mean()


In [9]:
dist_users_day=dist_users_day.groupby(['COD_PRO'],as_index=False).sum()

Same problem as before: some COD_PRO are missing. In this case we just remove the missing COD_PRO.

In [10]:
dic={}
for index, row in codist_ven.iterrows():
    dic[row.Origine]=row.Total_Mete

def prov_to_dist(p):
    try: 
        d=dic[p]
    except:
        d=-1
    return d 
prov_to_dist=np.vectorize(prov_to_dist)

In [11]:
dist_users_day['DISTANCES']=prov_to_dist(dist_users_day.COD_PRO.astype(int))
print(np.sum(dist_users_day.VISITORS[dist_users_day.DISTANCES==-1]))
dist_users_day=dist_users_day[dist_users_day.DISTANCES!=-1]

35980


In [12]:
dist_users_day

,COD_PRO,VISITORS,DISTANCES
0,1.0,39068,3.943948e+05
1,2.0,1940,3.382826e+05
2,3.0,9164,3.067730e+05
3,4.0,3372,4.339346e+05
4,5.0,1440,3.676197e+05
...,...,...,...
100,102.0,428,1.093870e+06
101,103.0,2000,3.541547e+05
105,108.0,30648,2.417719e+05
106,109.0,2676,3.961239e+05


### Make the distribution
Now we can compute the distribtion. We will do it for different number of bins.

In [13]:
def distribution(N,df):
    dist=df.DISTANCES.values
    freq=df.VISITORS.values

    bins=np.linspace(np.min(dist),np.max(dist),N+1)

    f_bins=[]
    for i in range(len(bins)-1):
        f=np.sum(freq[dist<=bins[i+1]])
        f-=np.sum(freq[dist<bins[i]])
        f_bins.append(f)

    bins=(bins[1:]+bins[:-1])*0.5

    return bins, f_bins

nbins=100

bins, f_bins = distribution(nbins,dist_users_day)

In [14]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=bins, y=f_bins, name='Distribution',
                        mode='lines+markers',
                        line=dict(color='royalblue', width=2, dash='dot')
                         ))

fig.update_layout(title="Distribution of visitors as a function of distance from Padua ",
                xaxis_title="Distance [m]",
                yaxis_title="Visitors")
fig.show()

As we can observe in the plot above, our distribution present lot of peaks. These peaks corresponds to 
the most popolous italian provincies as we can see in the next plot.

In [15]:
dist_users_day=dist_users_day[dist_users_day.COD_PRO!=28] #REMOVING PADOVA
nbins=180
bins, f_bins = distribution(nbins,dist_users_day)

big=['Milano','Venezia','Roma','Napoli','Bologna','Torino','Verona','Brescia','Firenze','Genova','Varese']
cod_big=[]
for i in big:
    cod_big.append(codist_prov[codist_prov.PROVINCIA==i].COD_PRO.values[0])

fig = go.Figure()


for i in range(len(big)):
    fig.add_trace(go.Scatter(x=[prov_to_dist(cod_big[i]),prov_to_dist(cod_big[i])], 
                            y=[min(f_bins),max(f_bins)],
                            mode='lines+text',
                            showlegend=False,
                            line=dict(color='red', width=2, dash='dot'),
                            opacity=.5,
                         ))

    fig.add_annotation(x=prov_to_dist(cod_big[i])+9e3, y=0.5*(min(f_bins)+max(f_bins)),
            text=big[i],
            showarrow=False,
            textangle=-90,
            font=dict(color='red',size=15))

fig.add_trace(go.Scatter(x=bins, y=f_bins, name='Distribution',
                        mode='lines+markers',
                        line=dict(color='royalblue', width=2, dash='dot'),
                        showlegend=False,
                        opacity=.7
                         ))



fig.update_xaxes(range=[0, 0.75*1e6])
fig.update_layout(title="Distribution of visitor as a function of distance from Padua ",
                xaxis_title="Distance [m]",
                yaxis_title="Visitors")
fig.show()

In [16]:
df=pd.DataFrame({})
b_df=np.array([])
x_df=np.array([])
y_df=np.array([])
for b in range(30,200,10):
    bins, f_bins = distribution(b,dist_users_day)
    f_bins=np.array(f_bins)

    x_df=np.concatenate((x_df,bins))
    y_df=np.concatenate((y_df,f_bins))

    b_df=np.concatenate((b_df,b*np.ones(len(bins))))

df['x']=x_df
df['y']=y_df
df['bins']=b_df.astype(int)

#df

In [17]:
import plotly.express as px

fig = px.line(df, x="x", y="y", animation_frame="bins",
                range_x=[min(df.x)-3e3,0.75*1e6],
                title="Distribution of visitor as a function of distance from Padua ")
fig.update_layout(title="Distribution of visitors as a function of distance from Padua ",
                xaxis_title="Distance [m]",
                yaxis_title="Visitors")
fig.show()

In the last plot we can observe that the distribution's shape changes with the number of bins.
As we increase the number of bins also increase the number of peaks. This beahaviour is due to 
the non-uniform distribution of the italian poulation that is more concentrated in the big cities.

In order to make a regression we will use a hight number of bins = 100 as the distribution's shape is 
quite stable after that value. Since the behavior is decreasing we will use a function : 
$
f(d)=\frac{A}{(d-B)^2} 
$



In [18]:
from scipy import optimize
def f(x,A,B):
    return A/((x-B)**2)

nbins=130
bins, f_bins = distribution(nbins,dist_users_day)
bins/=1e6
f_bins=np.array(f_bins)/1e6

max_params, params_covariance = optimize.curve_fit(f, bins, f_bins, p0=[0.03, -0.03])
x=np.linspace(np.min(bins),np.max(bins),200)

sigma_ab=np.square(np.diag(params_covariance))

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=f(x,max_params[0],max_params[1]), name='Fit',
                        mode='lines',
                        line=dict(color='orange', width=2, dash='dot'),
                        opacity=0.8
                         ))

fig.add_trace(go.Scatter(x=bins, y=f_bins, name='Distribution',
                        mode='markers',
                        marker=dict(color='blue',size=6),
                        opacity=0.5
                         ))

fig.update_layout(title="Fit of the previous distribution (nbins= "+str(nbins)+") ",
                xaxis_title="Distance [1e6 m]",
                yaxis_title="Visitors 1e6")
fig.show()
print(sigma_ab)

[1.32629823e-16 8.14415534e-11]


In [19]:
nbins=200
bins, f_bins = distribution(nbins,dist_users_day)
bins/=1e6
f_bins=np.array(f_bins)/1e6

x=np.linspace(np.min(bins),np.max(bins),200)

sigma_ab=np.square(np.diag(params_covariance))

fig = go.Figure()



fig.add_trace(go.Scatter(x=bins, y=f_bins, name='Distribution',
                        mode='markers',
                        #line=dict(color='blue', width=2, dash='dot'),
                        opacity=0.6
                         ))

fig.add_trace(go.Scatter(x=x, y=f(x,max_params[0],max_params[1]), name='Fit',
                        mode='lines',
                        line=dict(color='orange', width=5, dash='dot'),
                        opacity=1
                         ))

fig.update_layout(title="Distribution (nbins= "+str(nbins)+") and the previous fit",
                xaxis_title="Distance [1e6 m]",
                yaxis_title="Visitors 1e6")






fig.show()

As we can see even if we increase the number of bins the fit remains qulitatively valid. 

## Conclusions


As we can see in the plots there are points that are distant from our fit. 
As said before this discrepances 
should be due to the distribution of italian population that is not uniform but concentrated in the big cities.
In this view we can identify the points that stay above the fit as the big city and that even if are far 
from Padova have a large number of visitors; the points that stay under the fit as the areas near Padova that have 
a low concentration of population.

#### The next plot summarize these conlusions

In [20]:
nbins=200
bins, f_bins = distribution(nbins,dist_users_day)
bins/=1e6
f_bins=np.array(f_bins)/1e6

x=np.linspace(np.min(bins),np.max(bins),200)

sigma_ab=np.square(np.diag(params_covariance))

fig = go.Figure()



fig.add_trace(go.Scatter(x=bins, y=f_bins, name='Distribution',
                        mode='markers',
                        #line=dict(color='blue', width=2, dash='dot'),
                        opacity=0.6
                         ))

fig.add_trace(go.Scatter(x=x, y=f(x,max_params[0],max_params[1]), name='Fit',
                        mode='lines',
                        line=dict(color='orange', width=5, dash='dot'),
                        opacity=1
                         ))

fig.update_layout(title="Discrepances between fit and distribution (200 bins)",
                xaxis_title="Distance [1e6 m]",
                yaxis_title="Visitors 1e6")


fig.add_shape(type="circle",
    xref="x", yref="y",
    x0=0.07, y0=-0.01,
    x1=0.17, y1=0.03,
    opacity=0.2,
    fillcolor="red",
    line_color="red",
)

big=[['Milano',0.256,0.205],
    ['Roma',0.507,0.095],
    ['Napoli',.699,.026],
    ['Torino',.393,.041],
    ['Brescia',0.18,.1]]
    #['Venezia + Verona',.076,.62]]


for b in big:
    fig.add_annotation(x=b[1], y=b[2],
            text=b[0],
            showarrow=False,
            yshift=12,
            font=dict(color='red',size=15)
    )

fig.update_xaxes(range=[0, 0.75])

fig.add_annotation(x=0.1, y=.01,
            text="Under the fit",
            showarrow=True,
            arrowhead=2,
            font=dict(color='red',size=15),
            arrowcolor='red'
)


fig.show()